# Get data from parkrun website

In [ ]:
from functions.scrape import fetch_parkrun_data

Sometimes this function will result in  a 405 error. 

In that case you may need to go to the site and complete a captcha. 

Save the dataframe up to the stage it completed to .csv, then close and reload the notebook.

Continue to run the function from the next page to add to the final page with the existing dataframe also supplied as a parameter. This will concat the data to the existing dataframe.

### Function Description

The function scrapes data from the parkrun website. The website URL is in the format:

- **`{location}`**: The name of the parkrun location (e.g., 'brighton').
- **`{number}`**: The event number

The function takes 4 parameters:
- x: The event number to start from
- y: The event number to end at
- location: The parkrun location to extract data from. (If not supplied it will default to 'brighton')
- df: The dataframe to add to. (If not supplied it will create a new dataframe)

In [ ]:
df = fetch_parkrun_data(1, 200)

In [ ]:
df = fetch_parkrun_data(201, 771, df)

In [ ]:
df = fetch_parkrun_data(772, 826, df)

In [ ]:
df.to_csv('data/raw/parkrun_full_data.csv', index=False)